In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType
import pyspark.sql.functions as func

import matplotlib.pyplot as plt

In [2]:
sc = SparkContext('local')
sqlCtx = SQLContext( sc )

22/04/29 11:46:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = sqlCtx.read.csv('births.txt',inferSchema=True)
df.show()

+----+--------+--------+
| _c0|     _c1|     _c2|
+----+--------+--------+
|1880| 90993.0|110491.0|
|1881| 91954.0|100745.0|
|1882|107850.0|113688.0|
|1883|112321.0|104629.0|
|1884|129022.0|114445.0|
|1885|133055.0|107800.0|
|1886|144535.0|110784.0|
|1887|145982.0|101414.0|
|1888|178627.0|120853.0|
|1889|178366.0|110584.0|
|1890|190377.0|111025.0|
|1891|185482.0|101196.0|
|1892|212346.0|122037.0|
|1893|212906.0|112317.0|
|1894|222922.0|115772.0|
|1895|233630.0|117398.0|
|1896|237920.0|119570.0|
|1897|234202.0|112758.0|
|1898|258770.0|122693.0|
|1899|233023.0|106212.0|
+----+--------+--------+
only showing top 20 rows



In [7]:
df = df.withColumnRenamed('_c0','year').\     # \ 줄바꾸기
     withColumnRenamed('_c1','boys').\
withColumnRenamed('_c2','girls')

In [23]:
df.show()

+----+--------+--------+
|year|    boys|   girls|
+----+--------+--------+
|1880| 90993.0|110491.0|
|1881| 91954.0|100745.0|
|1882|107850.0|113688.0|
|1883|112321.0|104629.0|
|1884|129022.0|114445.0|
|1885|133055.0|107800.0|
|1886|144535.0|110784.0|
|1887|145982.0|101414.0|
|1888|178627.0|120853.0|
|1889|178366.0|110584.0|
|1890|190377.0|111025.0|
|1891|185482.0|101196.0|
|1892|212346.0|122037.0|
|1893|212906.0|112317.0|
|1894|222922.0|115772.0|
|1895|233630.0|117398.0|
|1896|237920.0|119570.0|
|1897|234202.0|112758.0|
|1898|258770.0|122693.0|
|1899|233023.0|106212.0|
+----+--------+--------+
only showing top 20 rows



## 연습문제

In [24]:
# 1. 2000년도 이후 데이터를 구하시요
df.where('year >= 2000').show()

+----+---------+---------+
|year|     boys|    girls|
+----+---------+---------+
|2000|1814601.0|1962406.0|
|2001|1799049.0|1941251.0|
|2002|1795206.0|1939815.0|
|2003|1825359.0|1973434.0|
|2004|1834145.0|1982794.0|
|2005|1845379.0|1994841.0|
|2006|1898463.0|2052377.0|
|2007|1919408.0|2072139.0|
|2008|1887234.0|2036289.0|
|2009|1832925.0|1979303.0|
|2010|1772738.0|1913851.0|
|2011|1753500.0|1893230.0|
+----+---------+---------+



In [25]:
# 2. 1995~2000 년도 데이터를 구하시요
df.where('year between 1995 and 2000').show()

+----+---------+---------+
|year|     boys|    girls|
+----+---------+---------+
|1995|1757927.0|1902832.0|
|1996|1752249.0|1893378.0|
|1997|1739806.0|1884122.0|
|1998|1765915.0|1910250.0|
|1999|1772632.0|1918809.0|
|2000|1814601.0|1962406.0|
+----+---------+---------+



In [15]:
# 3.  남아출생이 가장많은 top5를 구하시요(년도,남아수 출력)
df.orderBy('boys',ascending=False).select('year','boys').show(5)

+----+---------+
|year|     boys|
+----+---------+
|1957|2044160.0|
|1959|2023044.0|
|1960|2022093.0|
|1961|2017316.0|
|1958|2010884.0|
+----+---------+
only showing top 5 rows



In [28]:
# 4. 남아율(%)컬럼을 추가하고 각년도별 남아수가 차지하는 비율을 보여주시요
df.withColumn('boysPercnt', func.round( df['boys']/(df['boys']+df['girls'])*100,2)).show()

+----+--------+--------+----------+
|year|    boys|   girls|boysPercnt|
+----+--------+--------+----------+
|1880| 90993.0|110491.0|     45.16|
|1881| 91954.0|100745.0|     47.72|
|1882|107850.0|113688.0|     48.68|
|1883|112321.0|104629.0|     51.77|
|1884|129022.0|114445.0|     52.99|
|1885|133055.0|107800.0|     55.24|
|1886|144535.0|110784.0|     56.61|
|1887|145982.0|101414.0|     59.01|
|1888|178627.0|120853.0|     59.65|
|1889|178366.0|110584.0|     61.73|
|1890|190377.0|111025.0|     63.16|
|1891|185482.0|101196.0|      64.7|
|1892|212346.0|122037.0|      63.5|
|1893|212906.0|112317.0|     65.46|
|1894|222922.0|115772.0|     65.82|
|1895|233630.0|117398.0|     66.56|
|1896|237920.0|119570.0|     66.55|
|1897|234202.0|112758.0|      67.5|
|1898|258770.0|122693.0|     67.84|
|1899|233023.0|106212.0|     68.69|
+----+--------+--------+----------+
only showing top 20 rows



In [29]:
df.agg(func.sum('girls'), func.mean('girls')).show()

+------------+------------------+
|  sum(girls)|        avg(girls)|
+------------+------------------+
|1.64392696E8|1245399.2121212122|
+------------+------------------+



In [30]:
df.agg( func.sum('girls').alias('총합'), 
       func.mean('girls').alias('평균') ).show()

+------------+------------------+
|        총합|              평균|
+------------+------------------+
|1.64392696E8|1245399.2121212122|
+------------+------------------+

